# OVERVIEW

In this second part, as I just said, I will try to improve my results by building different cnn with differet characteristics. As I said before, my main goal is to improve test accuracy, first of all, I perform  data augmentation in order to have more data to train my network; we will see that just doing this stuff improves a lot our results. the details of data augmentation are described in the pdf and in the function dedicated to data aumentation, which is stored in a file call "utils". 



In [ ]:
#CELL DEDICATED TO IMPORT PACKAGES USEFULL FOR OUR PURPOSES
import numpy as np
import matplotlib.pyplot as plt
import PIL
from keras.models import Sequential
from keras.layers import Dense
import os
import cv2
from keras.utils import to_categorical
from keras import optimizers
from keras import initializers
from keras.callbacks import EarlyStopping
from keras.callbacks import History
from keras.layers.normalization import BatchNormalization
from keras.utils import plot_model
from sklearn.metrics import confusion_matrix
import time

from keras.layers import Conv2D,Flatten,Dropout, MaxPooling2D,AveragePooling2D

import import_ipynb
import utils



In [ ]:
# SOME IMPORTANT GLOBAL VARIABLE
labels = ["Bedroom","Coast","Forest","HighWay","Industrial","InsideCity","Kitchen","LivingRoom","Mountain","Office","OpenCountry","Store","Street","Suburb","TallBuilding"]
train_dir = '../images/train/'
test_dir = '../images/test/'

In [ ]:
#creation of  initial data
#train data
list_of_images = utils.list_of_path(labels,train_dir)
train_data,train_labels= utils.read_and_process_images(list_of_images)

#test data
list_of_images_test = utils.list_of_path(labels,test_dir)
test_data,test_labels = utils.read_and_process_images(list_of_images_test)


# DATA AUGMENTATION 

In order to try to achieve a better result, first of all we will try to augment our available data by performing a left-to-right tranformation and cropping. Details of how function "data_augmentation" works are in the presentation paper and in the function

In [ ]:
"""augmented train data with left-to-right reflection and cropping"""
train_data_aug,train_labels_aug = utils.data_augmentation(train_data,train_labels,cropping=True,crop_y=14,crop_x=14)

In [ ]:
"""reshape the data in order to make them feasible to cnn"""
train_data_aug = train_data_aug.reshape(train_data_aug.shape[0],train_data_aug.shape[1],train_data_aug.shape[2],1)
test_data = test_data.reshape(test_data.shape[0],test_data.shape[1],test_data.shape[2],1)

In [ ]:
"""create one-hot-encoding to make labels feasible for the cnn"""
test_labels_dummy = to_categorical(test_labels,15)
train_labels_dummy_aug = to_categorical(train_labels_aug,15)

# CREATION OF THE CNN 

FIrstly we will try to simply apply the same cnn that we build in the previus part of the exercise and see how the solution gets improved (remember: test_accuracy~0.33)

In [ ]:
sgd = optimizers.SGD(momentum=0.9,nesterov=True,decay = 1e-6, lr = 0.01)
norm = initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)

In [ ]:
model = Sequential()
#first convolutional layer
model.add(Conv2D(filters=8,kernel_size=3,padding = 'valid',activation='relu',input_shape=(64,64,1)))
model.add(MaxPooling2D(pool_size=2,strides=2))

#secondo convolutional layer
model.add(Conv2D(filters=16,kernel_size=3,strides=2,padding='valid',activation='relu',input_shape=(64,64,1)))
model.add(MaxPooling2D(pool_size=2,strides=2))

#third convolutional layer
model.add(Conv2D(filters=32,kernel_size=3,padding='valid',activation='relu',input_shape=(64,64,1)))
model.add(Flatten())
model.add(Dense(15,activation='relu',kernel_initializer=norm,bias_initializer='zeros'))
model.add(Dense(15,activation='softmax'))

#classification output
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics = ['accuracy'])

In [ ]:
history = History()
earlyStopping = EarlyStopping(min_delta=0.10,patience = 10)

start_time = time.time()
model.fit(train_data_aug,train_labels_dummy_aug,batch_size=32,epochs=100,validation_split=0.20,shuffle=True,callbacks=[earlyStopping,history])
end_time = time.time()

In [ ]:
model.evaluate(test_data,test_labels_dummy)

In [ ]:
print("time in seconds: ",end_time-start_time)

In [ ]:
v = model.predict_proba(test_data)

## PLOTTING

In [ ]:
plt.title("validation loss")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.plot(history.history['val_loss'])
plt.grid()
plt.savefig("images_point_two/val_loss_data_augmentation_only.jpg")

In [ ]:
plt.title("validation accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(history.history['val_accuracy'])
plt.grid()
plt.savefig("images_point_two/val_acc_data_augmentation_only.jpg")

In [ ]:
plt.title(" accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(history.history['accuracy'])
plt.grid()
plt.savefig("images_point_two/acc_data_augmentation_only.jpg")

In [ ]:
plt.title("loss")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(history.history['loss'])
plt.savefig("images_point_two/loss_data_augmentation_only.jpg")
plt.grid()

In [ ]:
from keras.utils import plot_model
print("My Custom CNN Network:")
plot_model(model, to_file='Images_point_two/custom-cnn.png', show_shapes=True, show_layer_names=True)

## OBSERVATION 

Only with the augmentation of data, we reach a better results in terms of test accuracy (we pass fro 0.33 to 0.44): what to do now? We can try to perform:

-BatchNormalization

-Add Dropout

-Change structure of the CNN, for example by introducing convolutional layer or by changing the size of filters


Our aim is to reach a results better than 0.60

In [ ]:
"""I wrote a generic function in which I preserve the main structure of the cnn built in the 
first part of the exercise but I add the possibility to add two dropout layer (choosing also
the dropout_rate) and 3 batch layer after performing relu """
adam = optimizers.Adam()
def cnn(optim,kernel_sz,batch = False, Drop = False, Dropout_rate = 0):
    """
    Function which creates a convolutional neural network
    
    Input optim = type of optimizer
    Input Kernel_sz = size of the kernel
    Input batch (False) = if True, it aplies batch normalization 
    Input Drop (False) = if True, it aplies dropout
    Input Dropout_rate (0) = rate of the dropout
    
    Output = convolutional neural network 
    """
    np.random.seed(9)
    model = Sequential()
    #first convolutional layer
    model.add(Conv2D(filters=8,kernel_size=kernel_sz,padding = 'valid',activation='relu',input_shape=(64,64,1)))
    if(batch ==True):
        model.add(BatchNormalization(axis = 1))
    model.add(MaxPooling2D(pool_size=2,strides=2))
    if(Drop==True):
        model.add(Dropout(Dropout_rate))

    #secondo convolutional layer
    model.add(Conv2D(filters=16,kernel_size=kernel_sz,strides=2,padding='valid',activation='relu',input_shape=(64,64,1)))
    if(batch==True):
        model.add(BatchNormalization(axis = 1))
    model.add(MaxPooling2D(pool_size=2,strides=2))
    if(Drop==True):
        model.add(Dropout(Dropout_rate))

    #third convolutional layer
    model.add(Conv2D(filters=32,kernel_size=kernel_sz,padding='valid',activation='relu',input_shape=(64,64,1)))
    model.add(Flatten())
    model.add(Dense(15,activation='relu',kernel_initializer=norm,bias_initializer='zeros'))
    if(batch==True):
        model.add(BatchNormalization(axis = 1))
    model.add(Dense(15,activation='softmax'))

    #classification output
    model.compile(loss='categorical_crossentropy',optimizer= optim,metrics = ['accuracy'])
    return model


def cnn_2():
    """
    Function which creates a simple cnn with no particular input (used in ensemble of network)
    
    Input = None
    
    Output = cnn
    """
    np.random.seed(9)
    model = Sequential()
    #first convolutional layer
    model.add(Conv2D(filters=8,kernel_size=(3,3),padding = 'valid',activation='relu',input_shape=(64,64,1)))
    model.add(BatchNormalization(axis = 1))
    model.add(MaxPooling2D(pool_size=2,strides=2))
    
    model.add(Dropout(0.3))

    #secondo convolutional layer
    model.add(Conv2D(filters=16,kernel_size=(3,3),strides=2,padding='valid',activation='relu',input_shape=(64,64,1)))
    model.add(BatchNormalization(axis = 1))
    model.add(MaxPooling2D(pool_size=2,strides=2))
    
    model.add(Dropout(0.3))

    #third convolutional layer
    model.add(Conv2D(filters=32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(64,64,1)))
    model.add(Flatten())
    model.add(Dense(15,activation='relu',kernel_initializer=norm,bias_initializer='zeros'))
    
    model.add(BatchNormalization(axis = 1))
    model.add(Dense(15,activation='softmax'))

    #classification output
    model.compile(loss='categorical_crossentropy',optimizer= adam,metrics = ['accuracy'])
    return model
    
    

In [ ]:
"""model_2: respect to the originale model, I just add dropout (with rate 0.5) and batchnormalization """
model_2 = cnn(adam,3,batch=True,Drop=True,Dropout_rate=0.5)

In [ ]:
history_2 = History()
earlyStopping_2 = EarlyStopping(min_delta=0.10,patience = 10)
start_time_2 = time.time()
model_2.fit(train_data_aug,train_labels_dummy_aug,batch_size=32,epochs=100,validation_split=0.20,shuffle=True,callbacks=[earlyStopping_2,history_2])
end_time_2 = time.time()

In [ ]:
"""Evaluation of the model"""
model_2.evaluate(test_data,test_labels_dummy)

In [ ]:
print("time in seconds: ",end_time_2-start_time_2)

In [ ]:
"""plotting the model"""
plot_model(model_2, to_file='Images_point_two/cnn-dropout-batch.png', show_shapes=True, show_layer_names=True)

In [ ]:
# cofunsion matrix of model with batch normalization and dropout 
plt.figure(figsize=(12,6)) 
y_pred = model.predict_classes(test_data)
cm = confusion_matrix(test_labels,y_pred)
utils.plot_confusion_matrix(cm,labels,"point_two_dropout_batch_cm","images_point_two/",normalize=True)

In [ ]:
plt.title("validation loss")
plt.xlabel("epochs")
plt.ylabel("val-loss")
plt.plot(history_2.history['val_loss'])
plt.grid()
plt.savefig("images_point_two/validation_loss_dropout_batch.jpg")

In [ ]:
plt.title("validation accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(history_2.history['val_accuracy'])
plt.grid()
plt.savefig("images_point_two/validation_accuracy_dropout_batch.jpg")

In [ ]:
plt.title(" accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(history_2.history['accuracy'])
plt.grid()
plt.savefig("images_point_two/accuracy_dropout_batch.jpg")

In [ ]:
plt.title("Loss")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(history_2.history['loss'])
plt.grid()
plt.savefig("images_point_two/loss_dropout_batch.jpg")

In [ ]:
"""change only the size of the kernels--> structures are always the same as the first convolutional network """
model_3 = cnn(adam,5,batch=True,Drop=True,Dropout_rate=0.3)

In [ ]:
history_3 = History()
earlyStopping_3 = EarlyStopping(min_delta=0.10,patience = 10)
start_time_3 = time.time()
model_3.fit(train_data_aug,train_labels_dummy_aug,batch_size=32,epochs=100,validation_split=0.15,shuffle=True,callbacks=[earlyStopping_3,history_3])
end_time_3 = time.time()

In [ ]:
model_3.evaluate(test_data,test_labels_dummy)


In [ ]:
print("time in seconds: ",end_time_3-start_time_3)

In [ ]:
plot_model(model_3,show_shapes=True, show_layer_names=True)

# ENSEMBLE OF NETWORK 

In the following cells I wrote two functions in order to perform the so called "ensemble of network".

-In the first function I train n times a cnn and I perform a prediction on test data in terms of probabilistic distribution along the 15 different classes.

-In the second class I perform the mean along the different obtained results

In [ ]:

def ensemble_of_network(mod,iteration,train_data, train_labels,test_data):
    """
    Function which performs ensemple of cnn model of the same type
    
    Input mod = function which gives a specific cnn
    Input iteration = number of model to be "ensembled"
    train_data = data for training the cnn
    train_labels = labels of the train data
    test_data = data to test and evaluate cnn
    
    Output prediction = array with prediction of test data in terms of probability, obtained with 
                        average
    Output pred_voting = array with prediction  
    Output validation_acc = all validation accuracy arrays of all models
    Output acc = all  accuracy arrays of all models
    Output validation_loss = all  validation_loss arrays of all models
    Output loss = all  loss arrays of all models
    """
    history = History()
    earlyStopping = EarlyStopping(min_delta=0.0,patience = 10)
    prediction = np.zeros((test_data.shape[0],15))
    loss = []
    validation_loss = []
    validation_acc = []
    acc = []
    pred_voting = []
    for i in range(iteration):
        model_temp = mod()
        model_temp.fit(train_data,train_labels,batch_size=32,epochs=100,validation_split=0.15,shuffle=True,callbacks=[earlyStopping,history])
        prediction = prediction + model_temp.predict_proba(test_data)
        pred_voting.append(model_temp.predict_classes(test_data))
        loss.append(history.history['loss'])
        validation_loss.append(history.history['val_loss'])
        validation_acc.append(history.history['val_accuracy'])
        acc.append(history.history['accuracy'])
    
    prediction = prediction/iteration
    
    return prediction,pred_voting,validation_acc,acc,validation_loss,loss
    
        
def calculate_accuracy(prediction,test_labels):
    """
    Function which calculate accuracy of the ensemble of network in case of average solution
    
    Input prediction = array with prediction of test data in terms of probability, obtained with 
                       average
    test_labels = array with true labels of test data
    
    Output res = accuracy 
    """
    total = test_labels.shape[0]
    cont = 0
    for i in range(len(prediction)):
        if(np.argmax(prediction[i])==test_labels[i]):
            cont = cont + 1
        else: 
            
            
            continue 
    res = cont/total
    return res 

##################################################################################
def determ_prediction(pred,test_lab):
    """
    function which determs final prediction with voting
    
    Input pred = predictions of ensemble of network
    Input test_data = test dataset
    
    Output res = array of final prediction for test data
    """
    res = np.zeros(len(pred[0]))
    for i in range(len(pred[0])):
        cont = np.zeros(15)
        #viaggio per le immagini
        for j in range(len(pred)):
            #viaggio per i tentativi di cnn
            
            cont[pred[j][i]]+=1
        
        maxi = np.argmax(cont)
        res[i] = maxi
        
    return res
    

def calulate_accuracy_with_voting(pred,test_labels):
    """
    Function which calculate accuracy if we apply voting
    
    Input pred = array of prediction 
    Input test_labels = array with true labels of test data
    
    Output res = accuracy
    """
    total = test_labels.shape[0]
    cont = 0
    for i in range(len(pred)):
        if(pred[i]==test_labels[i]):
            cont = cont +1
        else:
            continue
    res = cont/total
    return res
        


In [ ]:
"""Apply the algorithm which preforms the ensemble of network"""
ensembel_start_time = time.time()
pred,pred_vot,val_acc,val,val_loss,loss = ensemble_of_network(cnn_2,10,train_data_aug,train_labels_dummy_aug,test_data)
ensemble_end_time = time.time()

In [ ]:
calculate_accuracy(pred,test_labels)

In [ ]:
time_ensemble = convert(ensemble_end_time-ensembel_start_time)
time_ensemble

In [ ]:
plt.title("validation accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
for p in val_acc:
    plt.plot(p,linewidth=0.5)

plt.grid()

In [ ]:
plt.title("accuracy")
plt.xlabel("epochs")
plt.ylabel("accuracy")
for p in val:
    plt.plot(p,linewidth=0.5)
plt.grid()

In [ ]:
plt.title("validation loss")
plt.xlabel("epochs")
plt.ylabel("val-loss")
for p in val_loss:
    plt.plot(p,linewidth=0.5)
plt.grid()

In [ ]:
plt.title("Loss")
plt.xlabel("epochs")
plt.ylabel("accuracy")
for p in loss:
    plt.plot(p,linewidth=0.5)
plt.grid()

In [ ]:
res = determ_prediction(pred_vot,test_labels)


In [ ]:
acc = calulate_accuracy_with_voting(res,test_labels)

In [ ]:
acc

# play with number of layers

In this last part I simply do some "random" experiment, by adding some layer, both convolutional and dense, to my cnn

In [ ]:
np.random.seed(9)
model_4 = Sequential()
#first convolutional layer
model_4.add(Conv2D(filters=8,kernel_size=(3,3),strides = 1,padding = 'valid',activation='relu',input_shape=(64,64,1)))
model_4.add(BatchNormalization(axis=1))
model_4.add(MaxPooling2D(pool_size=2,strides=2))
model_4.add(Dropout(0.3))

#secondo convolutional layer
model_4.add(Conv2D(filters=16,kernel_size=(3,3),strides=1,padding='valid',activation='relu',input_shape=(64,64,1)))
model_4.add(BatchNormalization())
model_4.add(Dropout(0.3))
#third convolutional layer
model_4.add(Conv2D(filters=32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(64,64,1)))
model_4.add(BatchNormalization())
model_4.add(Dropout(0.3))

#add 1 convolutional layer
model_4.add(Conv2D(filters=64,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(64,64,1)))
model_4.add(BatchNormalization())
model_4.add(MaxPooling2D(pool_size=2,strides=2))
model_4.add(Dropout(0.3))


model_4.add(Flatten())
#add 1 dense layer

model_4.add(Dense(30,activation='relu',kernel_initializer=norm,bias_initializer='zeros'))
model_4.add(BatchNormalization())
model_4.add(Dense(15,activation='relu'))
model_4.add(BatchNormalization())
model_4.add(Dense(15,activation='softmax'))

#classification output
model_4.compile(loss='categorical_crossentropy',optimizer= adam,metrics = ['accuracy'])

In [ ]:
plot_model(model_4,show_shapes=True, show_layer_names=True)

In [ ]:
history_4 = History()
earlyStopping_4 = EarlyStopping(min_delta=0.10,patience = 10)
start_time_4 = time.time()
model_4.fit(train_data_aug,train_labels_dummy_aug,batch_size=32,epochs=100,validation_split=0.15,shuffle=True,callbacks=[earlyStopping_4,history_4])
end_time_4 = time.time()

In [ ]:
model_4.evaluate(test_data,test_labels_dummy)

In [ ]:
time_4 = convert(end_time_4-start_time_4)
time_4

In [ ]:
from keras.utils import plot_model
print("My Custom CNN Network:")
plot_model(model_4, to_file='Images_point_two/deeper_model.png', show_shapes=True, show_layer_names=True)